<br>

<img src="https://www.fedlex.admin.ch/assets/pictos/logo-ch.png" style="width:15%; float:right">

# Fedlex Linked Data Tutorial

## Einführung

Dieses Tutorial ist dazu gedacht, eine Einführung in Fedlex Linked Data zu geben. Dabei soll einerseits das verwendete Datenmodell vorgestellt werden und anderseits aufgezeigt werden, wie mit Hilfe von SPARQL Abfragen die Daten genutzt werden können. Fedlex ist die Publikationsplattform des schweizerischen Bundesrechts und baut sehr stark auf der Anwendung von Linked Data auf.

Dieses Tutorial ist ein sogenanntes **interaktives JupyterLite Notebook**. In diesem Notebook können Sie den Inhalt der einzelnen Zellen interaktiv ändern und diese Zellen direkt ausführen, um das Ergebnis Ihrer Änderungen sofort zu sehen. Die Zellen enthalten entweder [Markdown](https://en.wikipedia.org/wiki/Markdown)-Inhalt (wie diese Zelle) oder ausführbaren Python-Quellcode. Dies ist für ein Tutorial sehr hilfreich, weil Inhalte beliebig mit ausführbarem Quellcode kombiniert werden können. Es können also Abfragen gezeigt werden, diese erklärt werden und darauf weiter aufgebaut werden.

**Um direkt loslegen zu können klicken Sie oben im Menu auf Run -> Run All Cells.**  
**Einzelne ausgewählte Zellen können sie danach mit dem "Play-Button" erneut ausführen und so Abfragen individuell anpassen.**

Das Notebook startet mit einem [Setup](#Setup) der Programierumgebung. Das eigentliche Tutorial startet [hier](#Tutorial).

*Zusätzliche Informationen zu JupyterLite:*  
JupyterLite is ein spezielles Jupyter Notebook mit dem Vorteil, vollständig browserbasiert zu sein, ohne eine aufwändige Backend-Infrastruktur zu benötigen. Der Nachteil ist, dass die erstmalige Ausführung der Zellen einige Zeit in Anspruch nehmen kann, weil eine erhebliche Menge von Daten geladen werden muss. Nachfolgende Ausführungen werden aufgrund der gespeicherten Daten in Ihrem Browser-Cache viel schneller sein.

Wenn Sie mehr über die Handhabung von Jupyter Notebooks wissen wollen, finden Sie hier zwei nützliche Ressourcen:

- [Die JupyterLab Interface](https://jupyterlab.readthedocs.io/en/stable/user/interface.html)
- [Das Jupyter Notebook](https://jupyterlab.readthedocs.io/en/stable/user/notebook.html)

## Setup

Eine SPARQL Abfrage ist nichts anderes als ein POST-Request an den entsprechenden Triple Store Datenbank Server. Um diese Requests und die erhaltenen Antworten einfacher handhaben zu können, enthält dieses Notebook vorbereiteten Python Code, der nachfolgend importiert wird. Zusätzlich wird das `pandas` Modul importiert, welches die Möglichkeit bietet, die tabellarischen Daten der SPARQL Abfragen als [Pandas Dataframes](https://pandas.pydata.org/docs/index.html) weiter zu verarbeiten. 

In [1]:
import pandas as pd
from ext.sparql import query, display_result

# Tutorial

## Linked Data Einführung

Linked Data beschreibt ein **Framework für den Umgang mit Daten**, die sowohl für Menschen nützlich sein sollen, als auch maschinenlesbar sind inkl. einer von Computern verarbeitbaren Semantik. Also sowohl Menschen als auch Computer sollen die Daten "verstehen" und interpretieren können. 

### RDF

Das für Linked Data verwendete Datenformat ist RDF (Resource Description Framework). Das bedeutet, dass die Daten nicht als Tabellen (wie beispielsweise in relationalen Datenbanken) sondern als **Triples** abgespeichert werden. Triples folgen der grammatikalischen Struktur **Subjekt -> Prädikat -> Objekt** und können auch als grammatikalischer Satz verstanden werden. 

Die Information "**Der Apfel ist grün**" wird also mit dem Tripel **Apfel -> ist -> grün** ausgedrückt. Alle Teile eines Triples sind dabei durch weitere Eigenschaften definiert und beschrieben die wiederum in Form von Triples beschrieben sind. Diese vielseitigen Verknüpfungen führen zu einer Netzwerkstruktur, zu einem sogenannten Graphen.

Nachfolgendes Bild aus dem [W3C Primer für RDF](https://www.w3.org/TR/rdf11-primer/) veranschaulicht diese Zusammenhänge:

![](https://www.w3.org/TR/rdf11-primer/example-graph.jpg)

### URI

Eine weitere wichtige Eigenschaft von Linked Data ist, dass alle Teile eines Triples, also Subjekt, Prädikat und Objekt weltweit eindeutig identifizierbar sind. Dafür werden URI (Universal Resource Identifier) eingesetzt. Die URI https://fedlex.data.admin.ch/eli/cc/1999/404 beispielsweise ist der weltweit eindeutige Identifier für die Bundesverfassung in der systematischen Rechtssammlung. Typischerweise lassen sich URI **dereferenzieren**, das heisst, ein Request auf die entsprechende URI (bspw. in dem man sie in die Adresszeile eines Browsers eintippt) führt zu einer Website, die Infos zur entsprechenden URI enthält. Im Falle der URI der Bundesverfassung wird man auf eine Webpage weitergeleitet, die diverse Informationen zur Bundesverfassung (Chronologie, Änderungen, etc.) und den eigentlichen Text der Bundesverfassung enthält.

### Weitere Informationen zu Linked Data

Wer vertieft in das Thema Linked Data einsteigen möchte, dem sei beispielsweise [diese Youtube Playlist](https://www.youtube.com/watch?v=ON0wf0SEPx8&list=PLoOmvuyo5UAfY6jb46jCpMoqb-dbVewxg) empfohlen.

## Die Daten auf Fedlex

### Datenmodell JOLux

Linked Data und RDF definieren die Grundmechanismen der Datenspeicherung, das eigentliche Datenmodell (auch Ontologie oder Schema genannt) muss in der jeweiligen Anwendung festgelegt werden. Das von Fedlex verwendete Datenmodell heisst **JOLux**. Dieses stammt ursprünglich aus Luxemburg und wird inzwischen von der Schweiz und Luxemburg gemeinsam weiterentwickelt. JOLux definiert, welche Art von Objekten es gibt, wie diese in Beziehung stehen können und mit welchem Vokabular (also mit welchen Prädikaten und Objekten) diese bezeichnet werden. Weitere Informationen über das Datenmodell JOLux finden Sie [hier](https://fedlex.data.admin.ch/de-CH/home/models).

### Amtliche Sammlung

In der **Amtlichen Sammlung (AS)** (Englisch: Official Compilation (OC)) des Bundesrechts werden die neuen und geänderten Erlasse, Verträge und Beschlüsse chronologisch veröffentlicht. Typischerweise haben Beschlüsse die Form eines Mantelerlasses, in denen sowohl neue Gesetze erlassen werden, als auch schon bestehende geändert oder aufgehoben werden können. Jede Veröffentlichung in der amtlichen Sammlung erhält eine eindeutige AS-Nummer (z.B. [AS 2021 654](https://www.fedlex.admin.ch/eli/oc/2021/654)). Weitere Erläuterungen zur amtlichen Sammlung sind [hier](https://www.fedlex.admin.ch/de/oc/explanations-oc) zu finden.

### Systematische Rechtssammlung

Die **Systematische Rechtssammlung (SR)** (Englisch: Consolidated Compilation (CC)) des Bundesrechts stellt die konsolidierte Fassung des aktuell gültigen Rechts basierend auf der amtlichen Sammlung dar. Jeder Gesetzestext bekommt eine eindeutige SR-Nummer (z.B. [SR 101](https://www.fedlex.admin.ch/eli/cc/1999/404) für die Bundesverfassung), welche sich nicht auf eine bestimmte Version eines Gesetzes bezieht, sondern auf das Gesetz im Allgemeinen. Erscheint ein neuer Erlass in der AS der Teile des entsprechenden Gesetzes revidiert (z.B. Teile der Bundesverfassung durch [AS 2022 241](https://www.fedlex.admin.ch/eli/oc/2022/241)), so werden diese Änderungen in der systematischen Rechtssammlung konsolidiert und unter gleichbleibender SR-Nummer veröffentlicht (im Beispiel weiterhin SR 101). Weitere Erläuterungen zur systematischen Rechtssammlung sind [hier](https://www.fedlex.admin.ch/de/cc/explanations-cc) zu finden.

### Aufbau der URI bei Fedlex

URI können grundsätzlich sprechend oder nicht sprechend aufgebaut sein. Sprechend heisst in diesem Zusammenhang, dass die URI schon etwas über das entsprechende Objekt kodieren. Nicht sprechende URI sind typischerweise als beliebige Folgend von Zahlen und Buchstaben aufgebaut. Die URI von Fedlex sind sprechend, sie beginnen alle mit `https://fedlex.data.admin.ch/eli/`. Die Einträge für die AS beginnen mit `https://fedlex.data.admin.ch/eli/oc/` und diejenigen für die SR mit `https://fedlex.data.admin.ch/eli/cc/`. Der weitere Aufbau der URI ist [hier](https://fedlex.data.admin.ch/de-CH/home/convention) erklärt.

### Metadaten und Triples

Mit welchen Linked Data Triples ein bestimmter Eintrag in der AS oder SR in der Datenbank beschrieben ist, lässt sich über den [Metadaten-Explorer](https://fedlex.data.admin.ch/de-CH/metadata) anzeigen. Dazu müssen die entsprechenden URIs im Metadaten-Explorer eingegeben werden.

Die URI eines AS oder SR Eintrags lässt sich über die Web-Darstellung des entsprechenden Eintrags finden, indem man auf das Ketten-Symbol direkt neben "Alles einblenden" klickt:
<br><img src="img/sr101.jpg" width="800px">

[In den Metadaten-Explorer eingegeben](https://fedlex.data.admin.ch/de-CH/metadata?value=https://fedlex.data.admin.ch/eli/cc/1999/404) zeigt die oben genannte URI die abgespeicherten Eigenschaften der Bundesverfassung.
<br><img src="img/metadata.png" width="800px">

Um direkt den Metadaten-Explorer Eintrag für eine bestimmte URI zu erhalten, lässt sich die URI als Parameter in der URL zum Metadaten-Explorer hinzufügen, für die Bundesverfassung also: https://fedlex.data.admin.ch/de-CH/metadata?value=https://fedlex.data.admin.ch/eli/cc/1999/404

Fedlex ist so aufgebaut, dass die Dereferenzierung einer URI (also das Anzeigen der entsprechenden Website) mit einer von der URI abweichenden URL geschieht, die sich jedoch aus der URI ableiten lässt und auf die man bei Eingabe der URI in der Browser-Adresszeile auch automatisch weitergeleitet wird:

* URI: https://fedlex.data.admin.ch/eli/cc/1999/404
* URL (Dereferenzierung der URI für Deutsch): https://www.fedlex.admin.ch/eli/cc/1999/404/de

### Prefixes

URIs sind analog zu Webadressen aufgebaut. Um die Lesbarkeit zu verbessern, können Abkürzungen, sogenannte **Prefixes** definiert werden, die häufig genutzte Teile einer URI zusammenfassen, bspw. `fedlex` als Prefix für `https://fedlex.data.admin.ch/eli/`. Aus `https://fedlex.data.admin.ch/eli/cc/1999/404` wird dann `fedlex:cc/1999/404`.

Für die weiteren Teile des Tutorials definieren wir folgenden Prefixes:
* `fedlex` für `https://fedlex.data.admin.ch/eli/`; Namensraum für AS und SR Einträge
* `jolux` für `http://data.legilux.public.lu/resource/ontology/jolux#`; Vokabular für das JOLux Datenmodell
* `skos` für `http://www.w3.org/2004/02/skos/core#`; Externes Vokabular
* `rdf` für `http://www.w3.org/1999/02/22-rdf-syntax-ns#`; Externes Vokabular

Mehr zum Aufbau der URIs auf Fedlex finden Sie [hier](https://fedlex.data.admin.ch/de-CH/home/convention).

## Das Datenmodell erforschen und Daten abfragen

Um die gespeicherten Daten bspw. zur Bundesverfassung zu erhalten, gibt es drei Möglichkeiten:

- Die HTML Version (dereferenzierte URI) unter https://www.fedlex.admin.ch/eli/cc/1999/404/de
- Den Metadaten-Explorer unter https://fedlex.data.admin.ch/de-CH/metadata?value=https://fedlex.data.admin.ch/eli/cc/1999/404
- Die Abfrage der weiteren Daten mit SPARQL, einer Sprache zum Abfragen von Linked Data Datenbanken (so genannten Triple Stores), ähnlich zu SQL.

Die Website https://fedlex.admin.ch und alle dort angzeigten Daten basieren auf den Linked Data aus dem Triple Store. Informationen, die auf den HTML-Seiten von Fedlex zu sehen sind, sind also auch maschinenlesbar via SPARQL Abfrage verfügbar. Nachfolgend soll Schritt für Schritt das Datenmodell von Fedlex eingeführt werden und die entsprechenden SPARQL Queries aufgezeigt werden, um auf diese Daten maschinenlesbar zugreifen zu können.

### SPARQL <a class="anchor" id="SPARQL"></a>

SPARQL ist eine Query-Sprache für Linked Data Triple Stores. Für eine allgemeine Einführung in SPARQL siehe z.B.: https://jena.apache.org/tutorials/sparql.html

Abfragen (engl. Queries) können entweder direkt über das [SPARQL-Interface von Fedlex](https://fedlex.data.admin.ch/de-CH/sparql) eingegeben und ausgeführt werden, oder als HTTP-POST Request an den SPARQL-Endpoint (https://fedlex.data.admin.ch/sparqlendpoint) gesendet werden.

Die letztere Methode erlaubt es, eigene Anwendungen zu bauen, die automatisch aktuelle Daten von Fedlex abfragen können. Für dieses Tutorial verwenden wir diese Methode. Die Abfragen sind jedoch in beiden Fällen identisch. Für die Abfrage über das Interface kopieren sie einfach den Teil zwischen den `"""` und fügen sie im SPARQL-Interface ein.

### Pattern Matching

SPARQL Queries sind Aufträge an den Computer, bestimme Muster (Pattern) in den Daten zu finden (matching). Es können also mit Hilfe von SPARQL Muster vorgegeben werden, und die Datenbank gibt alle Triples zurück, die dieses Muster erfüllen. Einzelne Positionen der Triples können dabei bei einer Abfrage bewusst undefiniert gelassen und mit einer Variable bezeichnet werden. Variablen beginnen mit `?` und werden bei der Abfrage durch alle möglichen Elemente gefüllt, die dieses Pattern erfüllen. Eine ausführlicher Anleitung zum Pattern Matching ist [hier](https://programminghistorian.org/en/lessons/retired/graph-databases-and-SPARQL#rdf-in-brief) zu finden. 

### Die erste SPARQL Query

Wenn wir Infos zur Bundesverfassung wollen, können wir nach Triples suchen, in denen die Bundesverfassung als Subjekt erscheint. Dies geschieht mit folgender SPARQL Query:

In [2]:
df = await query("""

# SELECT beschreibt welche Variablen zurückgegeben werden sollen.
# Mit DISTINCT werden allfällige doppelte Ergebnisse aussortiert.

SELECT DISTINCT ?Prädikat ?Objekt WHERE {
    
    # Die Aussage beginnt mit der URI der Bundesverfassung als Subjekt, setzt das Prädikat und Objekt als Variablen und endet mit einem Punkt.
    
    <https://fedlex.data.admin.ch/eli/cc/1999/404> ?Prädikat ?Objekt .
} 

""", "F")

display_result(df)

,Prädikat,Objekt
0,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://data.legilux.public.lu/resource/ontology/jolux#Work
1,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://data.legilux.public.lu/resource/ontology/jolux#WorkAtOj
2,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://data.legilux.public.lu/resource/ontology/jolux#ConsolidationAbstract
3,http://data.legilux.public.lu/resource/ontology/jolux#basicAct,https://fedlex.data.admin.ch/eli/oc/1999/404
4,http://data.legilux.public.lu/resource/ontology/jolux#parliamentDraftId,1996.091
5,http://data.legilux.public.lu/resource/ontology/jolux#typeDocument,https://fedlex.data.admin.ch/vocabulary/resource-type/55
6,http://data.legilux.public.lu/resource/ontology/jolux#isRealizedBy,https://fedlex.data.admin.ch/eli/cc/1999/404/de
7,http://data.legilux.public.lu/resource/ontology/jolux#isRealizedBy,https://fedlex.data.admin.ch/eli/cc/1999/404/en
8,http://data.legilux.public.lu/resource/ontology/jolux#isRealizedBy,https://fedlex.data.admin.ch/eli/cc/1999/404/fr
9,http://data.legilux.public.lu/resource/ontology/jolux#isRealizedBy,https://fedlex.data.admin.ch/eli/cc/1999/404/it


Das Ergebniss ist eine Tabelle (eigentlich Pandas Dataframe) mit allen Prädikaten und den entsprechenden Objekten, die in allen abgespeicherten Triples mit der Bundesverfassung als Subjekt vorkommen. Während Prädikate immer URIs sind, können die Objekte auch sogeannte **Literals** sein, also reine Stringwerte (Zahlen und Buchstaben), die eine Information transportieren, aber die nicht dereferenzierbar sind. Solche Literals können beispielsweise Kalenderdaten sein oder Identifier oder Namen, etc.

Wir sehen, dass die Bundesverfassung vom `rdf:type` `jolux:ConsolidationAbstract` ist. Dieser Typ beschreibt Objekte, die einen SR-Eintrag darstellen, also ein Gesetz auf abstrakter Ebene repräsentieren.

### Expressions: Sprachversionen der SR Einträge

Da alle Texte in verschiedenen Sprachen existieren (und die URI https://fedlex.data.admin.ch/eli/cc/1999/404 der Bundesverfassung gibt ja keinen Hinweis auf eine bestimmte Sprache), müssen also innerhalb eines `jolux:ConsolidationAbstract` verschiedene Sprachversionen existieren. Diese sind vom `rdf:type` `jolux:Expression` und sind durch die Eigenschaft `jolux:isRealizedBy` mit dem sprachübergreifenden Eintrag des `jolux:ConsolidationAbstract` verknüpft.

Mit einem Klick auf die URI der deutschen Sprachversion (https://fedlex.data.admin.ch/eli/cc/1999/404/de) sehen wir, dass wir hier sowohl Titel, Abkürzung, als auch die SR Nummer auf deutsch finden. Diese URI beschreibt nicht die deutsche Sprache der eigentlichen Bundesverfassung sondern ist nur die Übersetzung der "Kopfdaten" der Bundesverfassung, also SR Nummer, Titel und Abkürzung. Der eigentliche Inhalt ist über die [Consolidations](#Consolidations:-Versionen-der-SR-Einträge) angebunden.

<br><br><img src=img/expression.svg>
<br><br>

Ausgehend von der URI der Bundesverfassung können wir diese Informationen also abfragen mit:

In [3]:
df = await query("""

# Definition eines Prefix zur besseren Lesbarkeit
PREFIX jolux: <http://data.legilux.public.lu/resource/ontology/jolux#>

SELECT DISTINCT ?SR_Nummer ?Titel ?Abkürzung WHERE {
    
    # Die erste Aussage wählt alle Expressions (Sprachversionen) aus die durch jolux:isRealizedBy mit der Bundesverfassung verknüpft sind.
    <https://fedlex.data.admin.ch/eli/cc/1999/404> jolux:isRealizedBy ?expression .
    
    # Die zweite Aussage grenzt die Expressions auf diejenige mit der Sprache Deutsch.
    ?expression jolux:language <http://publications.europa.eu/resource/authority/language/DEU> .
    
    # Die nächsten Aussagen fragen die gewünschen Daten ab, jeweils mit der Expression als Subjekt (möglich durch das Beenden der Aussage mit ; anstatt .)
    ?expression jolux:title ?Titel ;
                jolux:titleShort ?Abkürzung ; 
                jolux:historicalLegalId ?SR_Nummer .
} 

""", "F")

display_result(df)

,SR_Nummer,Titel,Abkürzung
0,101,Bundesverfassung der Schweizerischen Eidgenossenschaft vom 18. April 1999,BV


### Liste aller SR Einträge

Basierend auf dem was wir bereits gelernt haben können wir nun ganz einfach eine Liste aller SR-Einträge und die dazugehörigen Metadaten abfragen, indem wir die URI der Bundesverfassung durch eine Variable ersetzen die vom `rdf:type` `jolux:ConsolidationAbstract` sein soll (also einen SR Eintrag darstellt):

In [5]:
df = await query("""

# Definition von Prefixes zur besseren Lesbarkeit
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX jolux: <http://data.legilux.public.lu/resource/ontology/jolux#>

SELECT DISTINCT ?SR_Nummer ?Titel ?Abkürzung ?SR_URI WHERE {
    
    # Alle SR Einträge und ihre Expressions auswählen
    ?SR_URI rdf:type jolux:ConsolidationAbstract .
    ?SR_URI jolux:isRealizedBy ?expression .
    
    ?expression jolux:language <http://publications.europa.eu/resource/authority/language/DEU> .
    
    ?expression jolux:title ?Titel ;
                jolux:titleShort ?Abkürzung ; 
                jolux:historicalLegalId ?SR_Nummer .
} 

# Wir beschränken die Ausgabe auf die ersten 10 Einträge
LIMIT 10

""", "F")

display_result(df)

,SR_Nummer,Titel,Abkürzung,SR_URI
0,734.000,"Bundesgesetz vom 24. Juni 1902 betreffend die elektrischen Schwach- und Starkstromanlagen (Elektrizitätsgesetz, EleG)",EleG,https://fedlex.data.admin.ch/eli/cc/19/259_252_257
1,836.100,Bundesgesetz vom 20. Juni 1952 über die Familienzulagen in der Landwirtschaft (FLG),FLG,https://fedlex.data.admin.ch/eli/cc/1952/823_843_839
2,916.350,"Beschluss der Bundesversammlung vom 29. September 1953 über Milch, Milchprodukte und Speisefette (Milchbeschluss)",MB,https://fedlex.data.admin.ch/eli/cc/1953/1109_1132_1172
3,613.100,Bundesgesetz vom 19. Juni 1959 über den Finanzausgleich unter den Kantonen,FAG,https://fedlex.data.admin.ch/eli/cc/1959/931_961_953
4,831.201,Verordnung vom 17. Januar 1961 über die Invalidenversicherung (IVV),IVV,https://fedlex.data.admin.ch/eli/cc/1961/29_29_29
5,746.100,"Bundesgesetz vom 4. Oktober 1963 über Rohrleitungsanlagen zur Beförderung flüssiger oder gasförmiger Brenn- oder Treibstoffe (Rohrleitungsgesetz, RLG)",RLG,https://fedlex.data.admin.ch/eli/cc/1964/99_95_95
6,416.000,Bundesgesetz vom 19. März 1965 über die Gewährung von Beiträgen an die Aufwendungen der Kantone für Ausbildungsbeihilfen (Ausbildungsbeihilfengesetz),StipG,https://fedlex.data.admin.ch/eli/cc/1965/477_481_475
7,741.410,Verordnung vom 27. August 1969 über Bau und Ausrüstung der Strassenfahrzeuge (BAV),BAV,https://fedlex.data.admin.ch/eli/cc/1969/821_841_839
8,952.020,"Verordnung vom 17. Mai 1972 über die Banken und Sparkassen (Bankenverordnung, BankV)",BankV,https://fedlex.data.admin.ch/eli/cc/1972/821_832_752
9,514.511,Verordnung vom 10. Januar 1973 über das Kriegsmaterial,VKM,https://fedlex.data.admin.ch/eli/cc/1973/116_114_120


Der Übersicht halber wurde die Ausgabe mit dem Keyword `LIMIT` auf die ersten 10 Einträge begrenzt.

### Suche nach einer Abkürzung eines SR Eintrages

Falls nach dem Titel einer bestimmten Abkürzung gesucht werden will, ist das mit einer kleinen Anpassung für bspw. die Abkürzung "BankV" möglich. Es muss lediglich die Variable `?Abkürzung` durch den gesuchten Wert ersetzt werden. Da der gesuchte Wert ein Literal ist, muss er in Anführungszeichen gesetzt werden:

In [ ]:
df = await query("""

# Definition von Prefixes zur besseren Lesbarkeit
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX jolux: <http://data.legilux.public.lu/resource/ontology/jolux#>

SELECT DISTINCT ?SR_Nummer ?Titel ?SR_URI WHERE {
    
    # Alle SR Einträge und ihre Expressions auswählen
    ?SR_URI rdf:type jolux:ConsolidationAbstract .
    ?SR_URI jolux:isRealizedBy ?expression .
    
    ?expression jolux:language <http://publications.europa.eu/resource/authority/language/DEU> .
    
    ?expression jolux:title ?Titel ;
                jolux:titleShort "BankV" ; 
                jolux:historicalLegalId ?SR_Nummer .
} 

# Wir beschränken die Ausgabe auf die ersten 10 Einträge
LIMIT 10

""", "F")

display_result(df)

### Consolidations: Versionen der SR Einträge

Ein SR Eintrag vom `rdf:type` `jolux:ConsolidationAbstract` beschreibt einen Gesetzestext in der Systematischen Rechtssammlung auf abstrakter Ebene über die Zeit seines Bestehens hinweg. Die konkrete Version des konsolidierten Gesetzestextes zu einem bestimmten Zeitpunkt ist als ein Objekt vom `rdf:type` `jolux:Consolidation` abgespeichert und mit dem Prädikat `jolux:isMemberOf` mit dem entsprechenden SR Eintrag verknüpft (Beispiel: [Bundesverfassung Version vom 13.02.2022](https://fedlex.data.admin.ch/de-CH/metadata?value=https:%2F%2Ffedlex.data.admin.ch%2Feli%2Fcc%2F1999%2F404%2F20220213)).

<br><img src=img/consolidation.svg>
<br>

Ausgehend von der URI der Bundesverfassung können wir nun eine Liste der verschiedenen Versionen abfragen:

In [6]:
df = await query("""

# Definition von Abkürzungen zur besseren Lesbarkeit
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX jolux: <http://data.legilux.public.lu/resource/ontology/jolux#>

SELECT DISTINCT ?Consolidation_Datum ?Consolidation_URI WHERE {
    
    # Alle Consolidations auswählen die zur Bundesverfassung gehören
    ?Consolidation_URI jolux:isMemberOf <https://fedlex.data.admin.ch/eli/cc/1999/404> ;
                      # Abfragen des Datums des Inkrafttretens der speziefischen Version
                      jolux:dateApplicability ?Consolidation_Datum .
} 

# Wir ordnen die Ausgabe nach absteigendem Datum (neueste zuerst)
ORDER BY DESC(?Consolidation_Datum)

# Wir beschränken die Ausgabe auf die ersten 10 Einträge
LIMIT 10

""", "F")

display_result(df)

,Consolidation_Datum,Consolidation_URI
0,2024-01-01,https://fedlex.data.admin.ch/eli/cc/1999/404/20240101
1,2022-02-13,https://fedlex.data.admin.ch/eli/cc/1999/404/20220213
2,2021-11-28,https://fedlex.data.admin.ch/eli/cc/1999/404/20211128
3,2021-03-07,https://fedlex.data.admin.ch/eli/cc/1999/404/20210307
4,2021-01-01,https://fedlex.data.admin.ch/eli/cc/1999/404/20210101
5,2020-01-01,https://fedlex.data.admin.ch/eli/cc/1999/404/20200101
6,2018-09-23,https://fedlex.data.admin.ch/eli/cc/1999/404/20180923
7,2018-01-01,https://fedlex.data.admin.ch/eli/cc/1999/404/20180101
8,2017-09-24,https://fedlex.data.admin.ch/eli/cc/1999/404/20170924
9,2017-02-12,https://fedlex.data.admin.ch/eli/cc/1999/404/20170212


Mit dem Befehl `ORDER BY` wurde die Ausgabe nach Datum sortiert.

### Manifestations: Dateiformate und Datenzugriff <a class="anchor" id="manifestations"></a>

Jede Version (`jolux:Consolidation`) eines SR-Eintrags (`jolux:ConsolidationAbstract`) hat dabei wiederum verschiedene Sprachversionen (`jolux:Expression`). Rufen wir die Metadaten einer solchen ab (Beispiel: [Bundesverfassung in Deutsch, Version vom 13.02.2022](https://fedlex.data.admin.ch/eli/cc/1999/404/20220213/de)), sehen wir, dass die `jolux:Expression` durch `jolux:isEmbodiedBy` wiederum mit verschiedenen Objekten vom `rdf:type` `jolux:Manifestation` verknüpft ist. Diese repräsentieren die verschiedenen Dateiformate (XML, PDF, HTML, docx, doc) in denen der Text zur Verfügung steht und zeigen ihrerseits mit dem Prädikat `jolux:isExemplifiedBy` auf den dazugehörigen Downloadlink für das Dokument.

<br><img src=img/manifestation.svg>
<br>

Mit dem Wissen über Textversionen, Sprachversionen, und Dateiformate können wir nun die vorherige Abfrage ergänzen und uns zusätzlich den Link zu dem jeweiligen PDF der verschiedenen deutschen Versionen der Bundesverfassung ausgeben lassen:

In [7]:
df = await query("""

# Definition von Abkürzungen zur besseren Lesbarkeit
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX jolux: <http://data.legilux.public.lu/resource/ontology/jolux#>

SELECT DISTINCT ?Consolidation_Datum ?Consolidation_URI ?PDF_Link WHERE {
    
    ?Consolidation_URI jolux:isMemberOf <https://fedlex.data.admin.ch/eli/cc/1999/404> ;
                      jolux:dateApplicability ?Consolidation_Datum ;
                      # Wähle alle Sprachversionen der verschiedenen Textversionen
                      jolux:isRealizedBy ?Expression .
                      
    # Reduziere die Auswahl auf die Deutschen Sprachversionen
    ?Expression jolux:language <http://publications.europa.eu/resource/authority/language/DEU> ;
                # Wähle alle verfügbaren Dateiformate der Sprachversion
                jolux:isEmbodiedBy ?Manifestation .
    
    # Reduziere die Auswahl auf Einträge mit dem Datenformat PDF
    ?Manifestation jolux:format <http://publications.europa.eu/resource/authority/file-type/PDF>;
                   # Rufe den Download-Link zum entsprechenden Dokument ab
                   jolux:isExemplifiedBy ?PDF_Link

} 

# Wir ordnen die Ausgabe nach absteigendem Datum (neueste zuerst)
ORDER BY DESC(?Consolidation_Datum)

# Wir beschränken die Ausgabe auf die ersten 10 Einträge
LIMIT 10

""", "F")

display_result(df)

,Consolidation_Datum,Consolidation_URI,PDF_Link
0,2024-01-01,https://fedlex.data.admin.ch/eli/cc/1999/404/20240101,https://fedlex.data.admin.ch/filestore/fedlex.data.admin.ch/eli/cc/1999/404/20240101/de/pdf-a/fedlex-data-admin-ch-eli-cc-1999-404-20240101-de-pdf-a.pdf
1,2022-02-13,https://fedlex.data.admin.ch/eli/cc/1999/404/20220213,https://fedlex.data.admin.ch/filestore/fedlex.data.admin.ch/eli/cc/1999/404/20220213/de/pdf-a/fedlex-data-admin-ch-eli-cc-1999-404-20220213-de-pdf-a-4.pdf
2,2021-11-28,https://fedlex.data.admin.ch/eli/cc/1999/404/20211128,https://fedlex.data.admin.ch/filestore/fedlex.data.admin.ch/eli/cc/1999/404/20211128/de/pdf-a/fedlex-data-admin-ch-eli-cc-1999-404-20211128-de-pdf-a-2.pdf
3,2021-03-07,https://fedlex.data.admin.ch/eli/cc/1999/404/20210307,https://fedlex.data.admin.ch/filestore/fedlex.data.admin.ch/eli/cc/1999/404/20210307/de/pdf-a/fedlex-data-admin-ch-eli-cc-1999-404-20210307-de-pdf-a-3.pdf
4,2021-01-01,https://fedlex.data.admin.ch/eli/cc/1999/404/20210101,https://fedlex.data.admin.ch/filestore/fedlex.data.admin.ch/eli/cc/1999/404/20210101/de/pdf-a/fedlex-data-admin-ch-eli-cc-1999-404-20210101-de-pdf-a-2.pdf
5,2020-01-01,https://fedlex.data.admin.ch/eli/cc/1999/404/20200101,https://fedlex.data.admin.ch/filestore/fedlex.data.admin.ch/eli/cc/1999/404/20200101/de/pdf-a/fedlex-data-admin-ch-eli-cc-1999-404-20200101-de-pdf-a.pdf
6,2018-09-23,https://fedlex.data.admin.ch/eli/cc/1999/404/20180923,https://fedlex.data.admin.ch/filestore/fedlex.data.admin.ch/eli/cc/1999/404/20180923/de/pdf-a/fedlex-data-admin-ch-eli-cc-1999-404-20180923-de-pdf-a.pdf
7,2018-01-01,https://fedlex.data.admin.ch/eli/cc/1999/404/20180101,https://fedlex.data.admin.ch/filestore/fedlex.data.admin.ch/eli/cc/1999/404/20180101/de/pdf-a/fedlex-data-admin-ch-eli-cc-1999-404-20180101-de-pdf-a.pdf
8,2017-09-24,https://fedlex.data.admin.ch/eli/cc/1999/404/20170924,https://fedlex.data.admin.ch/filestore/fedlex.data.admin.ch/eli/cc/1999/404/20170924/de/pdf-a/fedlex-data-admin-ch-eli-cc-1999-404-20170924-de-pdf-a.pdf
9,2017-02-12,https://fedlex.data.admin.ch/eli/cc/1999/404/20170212,https://fedlex.data.admin.ch/filestore/fedlex.data.admin.ch/eli/cc/1999/404/20170212/de/pdf-a/fedlex-data-admin-ch-eli-cc-1999-404-20170212-de-pdf-a.pdf
